This notebook enables the user to reproduce the analysis steps we have performed for our study *They crossed the valley of Catamarca: A study of narrative space in novel openings* based on the annotation data saved as [Catma](https://www.catma.de) project in `/CATMA_4AA4ADC0-4C28-54F9-B6A1-5DCEFF34B90B_DH2025_CANSpiN` and as `.tsv` data in `/canspin-deu-19`, `/canspin-deu-20`, `/canspin-lat-19`, and `/canspin-spa-19`.

If you wish to see the analysis results only, it is not necessary to execute this notebook. In this case, see our paper and the content of the `/results` folder.

To use the notebook, install the [gitma-canspin package](https://github.com/CANSpiNproject/gitma-canspin/tree/v1.6.2) first, following the instructions of its README.

## initialization

In [1]:
# imports
from gitma_canspin.canspin import AnnotationAnalyzer

import pandas as pd
import plotly
import math
import json
import os
import itertools

from typing import Tuple

In [2]:
# load the analyzer with the catma project from the CATMA_4AA4ADC0-4C28-54F9-B6A1-5DCEFF34B90B_DH2025_CANSpiN folder
analyzer = AnnotationAnalyzer(init_settings={'project_name': 'CATMA_4AA4ADC0-4C28-54F9-B6A1-5DCEFF34B90B_DH2025_CANSpiN'})

gitma_canspin.project - INFO - Loading tagsets ...
gitma_canspin.project - INFO - 	Found 3 tagset(s).
gitma_canspin.project - INFO - Loading documents ...
gitma_canspin.project - INFO - 	Found 6 document(s).
gitma_canspin.project - INFO - Loading annotation collections ...
gitma_canspin.project - INFO - 	Found 6 annotation collection(s).
gitma_canspin.project - INFO - 	Annotation collection "Collection CS1 v1.1.0 - Gold Standard" for document "El pozo del Yocci"
gitma_canspin.project - INFO - 		Annotations: 201
gitma_canspin.project - INFO - 	Annotation collection "CS1 v1.1.0 - Nils (Gold: 1)" for document "El Señor de Bembibre"
gitma_canspin.project - INFO - 		Annotations: 285
gitma_canspin.project - INFO - 	Annotation collection "CS1 v1.1.0 - Ulrike (Gold standard: 1)" for document "CANSpiN-spa-19-008"
gitma_canspin.project - INFO - 		Annotations: 1332
gitma_canspin.project - INFO - 	Annotation collection "Nils -- CS1 1.1.0 (Gold: 1-1-1)" for document "DEU-19_001"
gitma_canspin.proje

In [3]:
# display loaded tsv annotations from corpus folders
analyzer.print_tsv_annotations_overview()

gitma_canspin.canspin - INFO - tsv files found in canspin project!



overview:
- schema "cs1"
	CANSpiN-deu-19_001_1-1-1.tsv
	CANSpiN-deu-19_030_1-1-1.tsv
	CANSpiN-deu-20_002_1_shuffled.tsv
	CANSpiN-deu-20_021_1_shuffled.tsv
	CANSpiN-lat-19_004_1.tsv
	CANSpiN-lat-19_041_1.tsv
	CANSpiN-spa-19_001_1.tsv
	CANSpiN-spa-19_008_1.tsv


In [4]:
# display loaded annotation collections from catma project
analyzer.print_projects_annotation_collection_list()


Annotation collection list:
index	collection_name	text_title
0	Collection CS1 v1.1.0 - Gold Standard	El pozo del Yocci
1	CS1 v1.1.0 - Nils (Gold: 1)	El Señor de Bembibre
2	CS1 v1.1.0 - Ulrike (Gold standard: 1)	CANSpiN-spa-19-008
3	Nils -- CS1 1.1.0 (Gold: 1-1-1)	DEU-19_001
4	Nils -- CS1 V.1.1.0 (Gold:1-1-1)	DEU-19_030
5	Collection CS1 v1.1.0 - Gold Standard	El falso Inca


## steps
Perform steps 1 and 2 in the specified order to obtain a correct result. Step 3 can be executed independently.

In every step, the result can be presented in a cell or as temporary file.
In addition the output can be saved to files into the `perform_analysis_output` folder. Use the following cell to control the show and save to file features.

- [1. get CS1 annotation statistics (as dict / JSON)](#get-CS1-annotation-statistics-as-JSON-file)
- [2. get bar charts with annotation amounts of all chapters normalized to their token amount (as HTML)](#get-bar-charts-with-annotation-amounts-of-all-chapters-normalized-to-their-token-amount)
- [3. get first character event overview (as PNG file)](#get-first-character-event-overview)
- [4. get first character event cs1 relations (as PNG file)](#get-first-character-event-cs1-relations)

In [ ]:
# activate or deactivate show result in cell
get_cs1_annotation_statistics__show_dict = False
get_bar_charts_with_annotation_amounts_of_all_chapters__show_html = False
get_first_character_event_overview__show_temporary_image = False

# activate or deactivate save to file features
get_cs1_annotation_statistics__save_to_json = True
get_bar_charts_with_annotation_amounts_of_all_chapters__save_to_html = True
get_first_character_event_overview__save_to_png = True

In [6]:
# further settings

# additional token selection for steps 1 and 2 (in addition to the mandatory complete token selection)
token_selection: Tuple[int, int] = (0, 1000)

#### get CS1 annotation statistics as JSON file

In [7]:
# get the annotation statistics from the tsv files (for the entire chapters and for the first tokens respectively)
# and translate the class names into English

results: dict = {
    'whole_chapters': analyzer.get_corpus_annotation_statistics(),
    'first_1000_token': analyzer.get_corpus_annotation_statistics({
        'calculations': {
            'amount_of_annotations': True,
            'amount_of_annotations_by_class': True,
            'amount_of_token': True,
            'amount_of_annotated_token': True,
            'amount_of_annotated_token_by_class': True,
            'ratios': True,
            'word_lists_by_class': True
        },
        'custom_grouping': None,
        'text_borders': token_selection
    })
}

key_translation = {
    'Ort-Container': 'Place-Container',
    'Ort-Container-BK': 'Place-Container-MC',
    'Ort-Objekt': 'Place-Object',
    'Ort-Objekt-BK': 'Place-Object-MC',
    'Ort-Abstrakt': 'Place-Abstract',
    'Ort-Abstrakt-BK': 'Place-Abstract-MC',
    'Ort-ALT': 'Place-ALT',
    'Bewegung-Subjekt': 'Movement-Subject',
    'Bewegung-Objekt': 'Movement-Object',
    'Bewegung-Licht': 'Movement-Light',
    'Bewegung-Schall': 'Movement-Sound',
    'Bewegung-Geruch': 'Movement-Smell',
    'Bewegung-ALT': 'Movement-ALT',
    'Dimensionierung-Groesse': 'Dimensioning-Size',
    'Dimensionierung-Abstand': 'Dimensioning-Distance',
    'Dimensionierung-Menge': 'Dimensioning-Amount',
    'Dimensionierung-ALT': 'Dimensioning-ALT',
    'Positionierung': 'Positioning',
    'Positionierung-ALT': 'Positioning-ALT',
    'Richtung': 'Direction',
    'Richtung-ALT': 'Direction-ALT',
    'Lugar-Contenedor': 'Place-Container',
    'Lugar-Contenedor-CM': 'Place-Container-MC',
    'Lugar-Objeto': 'Place-Object',
    'Lugar-Objeto-CM': 'Place-Object-MC',
    'Lugar-Abstracto': 'Place-Abstract',
    'Lugar-Abstracto-CM': 'Place-Abstract-MC',
    'Lugar-ALT': 'Place-ALT',
    'Movimiento-Sujeto': 'Movement-Subject',
    'Movimiento-Objeto': 'Movement-Object',
    'Movimiento-Luz': 'Movement-Light',
    'Movimiento-Sonido': 'Movement-Sound',
    'Movimiento-Olfato': 'Movement-Smell',
    'Movimiento-ALT': 'Movement-ALT',
    'Dimensionamiento-Tamaño': 'Dimensioning-Size',
    'Dimensionamiento-Distancia': 'Dimensioning-Distance',
    'Dimensionamiento-Cantitad': 'Dimensioning-Amount',
    'Dimensionamiento-ALT': 'Dimensioning-ALT',
    'Posicionamiento': 'Positioning',
    'Posicionamiento-ALT': 'Positioning-ALT',
    'Dirección': 'Direction',
    'Dirección-ALT': 'Direction-ALT'
}

def merge_word_list_by_class_dicts(first: dict, second: dict) -> dict:
    result = first
    for token, token_amount in second.items():
        if token not in result:
            result[token] = token_amount
            continue
        result[token] = result[token] + token_amount
    result = dict(sorted(result.items(), key=lambda x: int(x[1]), reverse=True))
    return result

def translate_dict(input: dict, translation: dict) -> dict:
    translated = {}
    if len([key for key in input if key in key_translation]) == len(key_translation):
        # for translating schema totals per class with mixed german and spanish classes...
        if isinstance(input[list(key_translation.keys())[0]], int):
            # ...in case of class instances amounts
            for key, value in input.items():
                translated_key: str = key_translation[key]
                translated[translated_key] = value if not translated.get(translated_key) else translated[translated_key] + value
            translated = dict(sorted(translated.items(), key=lambda x: int(x[1]), reverse=True))
        elif isinstance(input[list(key_translation.keys())[0]], dict):
            # ...in case of word lists
            for key, value in input.items():
                translated_key: str = key_translation[key]
                translated[translated_key] = value if not translated.get(translated_key) else merge_word_list_by_class_dicts(translated[translated_key], value)
    else:
        # for translating everything else
        translated = dict([(translation.get(k, k), v) for k, v in input.items()])
    for key, value in translated.items():
        if isinstance(value, dict):
            translated[key] = translate_dict(value, translation)
    return translated

for result_type in results:
    results[result_type] = translate_dict(input=results[result_type], translation=key_translation)


In [8]:
# print annotation statistics

if get_cs1_annotation_statistics__show_dict:
    for result_type in results:
        print(
            json.dumps(
                results[result_type],
                indent=2, 
                sort_keys=False, 
                ensure_ascii=False
            )
        )


In [9]:
# safe annotation statistics to files

if get_cs1_annotation_statistics__save_to_json:
    for result_type in results:
        filename: str = f'annotation_statistics__{result_type}.json'
        filepath: str = os.path.join('perform_analysis_output', filename)
        json_file_str: str = json.dumps(results[result_type], indent=2, sort_keys=False, ensure_ascii=False)

        if not (os.path.isdir('perform_analysis_output')):
            os.makedirs('perform_analysis_output')

        if (os.path.isfile(filepath)):
            print(f'JSON file {filepath} already exists and will be overwritten.')

        with open(filepath, 'w+') as file:
            file.write(json_file_str)
            print(f'JSON file {filepath} successfully created.')

JSON file perform_analysis_output\annotation_statistics__whole_chapters.json successfully created.
JSON file perform_analysis_output\annotation_statistics__first_1000_token.json successfully created.


#### get bar charts with annotation amounts of all chapters normalized to their token amount

In [10]:
# for annotation data with all tokens
# you may manipulate and save the plotly diagram as png image file by using the interface of the html output

data_dict: dict = {
    'text': list(itertools.chain(*[[item] * 21 for item in [
        'DEU19_030', 
        'DEU19_001', 
        'DEU20_002', 
        'DEU20_021', 
        'SPA19_001', 
        'SPA19_008', 
        'LAT19_004', 
        'LAT19_041'
    ]])),
    'annotation_class': [
        'Place-Container', 
        'Place-Container-MC', 
        'Place-Object', 
        'Place-Object-MC', 
        'Place-Abstract', 
        'Place-Abstract-MC', 
        'Place-ALT',
        'Movement-Subject',
        'Movement-Object',
        'Movement-Light',
        'Movement-Sound',
        'Movement-Smell',
        'Movement-ALT',
        'Dimensioning-Size',
        'Dimensioning-Distance',
        'Dimensioning-Amount',
        'Dimensioning-ALT',
        'Positioning',
        'Positioning-ALT',
        'Direction',
        'Direction-ALT'
    ] * 8,
    'amount': []
}
data_dict['amount'] = list(itertools.chain(*[[results['whole_chapters']['ratios']['cs1'][corpus_file[0]][corpus_file[1]]['annotations_by_class_in_file:total_token_amount_in_file'].get(annotation_class, 0) * 100] for corpus_file in [
    ('canspin-deu-19', 'CANSpiN-deu-19_030_1-1-1.tsv'), 
    ('canspin-deu-19', 'CANSpiN-deu-19_001_1-1-1.tsv'),
    ('canspin-deu-20', 'CANSpiN-deu-20_002_1_shuffled.tsv'),
    ('canspin-deu-20', 'CANSpiN-deu-20_021_1_shuffled.tsv'),
    ('canspin-spa-19', 'CANSpiN-spa-19_001_1.tsv'),
    ('canspin-spa-19', 'CANSpiN-spa-19_008_1.tsv'),
    ('canspin-lat-19', 'CANSpiN-lat-19_004_1.tsv'),
    ('canspin-lat-19', 'CANSpiN-lat-19_041_1.tsv')
] for annotation_class in data_dict['annotation_class'][:21]]))

data: pd.DataFrame = pd.DataFrame(data_dict)

figure: plotly.graph_objects.Figure = plotly.express.bar(
    data_frame=data,
    x='text',
    y='amount',
    color='annotation_class',
    labels={
        "text": "texts",
        "amount": "annotation amount (in %)",
        "annotation_class": "annotation classes"
    },
    title='CS1 annotation amounts inside the initial chapters with all tokens <br><sub>(in percentage of the total token amount of the respective chapter)</sub>',
    color_discrete_map={
        'Place-Container': '#B6D3FF',
        'Place-Container-MC': '#CCDEFF',
        'Place-Object': '#D4EAFF',
        'Place-Object-MC': '#E6F2FF',
        'Place-Abstract': '#89A8F6',
        'Place-Abstract-MC': '#98C3FA',
        'Place-ALT': '#90A6C7',
        'Movement-Subject': '#FF6D6D',
        'Movement-Object': '#F60D00',
        'Movement-Sound': '#FF4949',
        'Movement-Light': '#CA0B0B',
        'Movement-Smell': '#B60000',
        'Movement-ALT': '#960000',
        'Direction': '#92FFBD',
        'Direction-ALT': '#75CC96',
        'Positioning': '#DB8300',
        'Positioning-ALT': '#B56A01',
        'Dimensioning-Distance': '#8AB6AD',
        'Dimensioning-Size': '#7CD3C0',
        'Dimensioning-Amount': '#7EF5D9',
        'Dimensioning-ALT': '#60847B'
    },
    width=1400,
    height=800
)

figure.update_layout(font={'size': 18})

if get_bar_charts_with_annotation_amounts_of_all_chapters__show_html:
    figure.show()

if get_bar_charts_with_annotation_amounts_of_all_chapters__save_to_html:
    if not (os.path.isdir('perform_analysis_output')):
        os.makedirs('perform_analysis_output')
    figure.write_html(os.path.join('perform_analysis_output', 'cs1_annotation_amounts__all_tokens.html'))


In [11]:
# for annotation data with a user defined amount of tokens
# you may manipulate and save the plotly diagram as png image file by using the interface of the html output

data_dict: dict = {
    'text': list(itertools.chain(*[[item] * 21 for item in [
        'DEU19_030', 
        'DEU19_001', 
        'DEU20_002', 
        'DEU20_021', 
        'SPA19_001', 
        'SPA19_008', 
        'LAT19_004', 
        'LAT19_041'
    ]])),
    'annotation_class': [
        'Place-Container', 
        'Place-Container-MC', 
        'Place-Object', 
        'Place-Object-MC', 
        'Place-Abstract', 
        'Place-Abstract-MC', 
        'Place-ALT',
        'Movement-Subject',
        'Movement-Object',
        'Movement-Light',
        'Movement-Sound',
        'Movement-Smell',
        'Movement-ALT',
        'Dimensioning-Size',
        'Dimensioning-Distance',
        'Dimensioning-Amount',
        'Dimensioning-ALT',
        'Positioning',
        'Positioning-ALT',
        'Direction',
        'Direction-ALT'
    ] * 8,
    'amount': []
}
data_dict['amount'] = list(itertools.chain(*[[results['first_1000_token']['ratios']['cs1'][corpus_file[0]][corpus_file[1]]['annotations_by_class_in_file:total_token_amount_in_file'].get(annotation_class, 0) * 100] for corpus_file in [
    ('canspin-deu-19', 'CANSpiN-deu-19_030_1-1-1.tsv'), 
    ('canspin-deu-19', 'CANSpiN-deu-19_001_1-1-1.tsv'),
    ('canspin-deu-20', 'CANSpiN-deu-20_002_1_shuffled.tsv'),
    ('canspin-deu-20', 'CANSpiN-deu-20_021_1_shuffled.tsv'),
    ('canspin-spa-19', 'CANSpiN-spa-19_001_1.tsv'),
    ('canspin-spa-19', 'CANSpiN-spa-19_008_1.tsv'),
    ('canspin-lat-19', 'CANSpiN-lat-19_004_1.tsv'),
    ('canspin-lat-19', 'CANSpiN-lat-19_041_1.tsv')
] for annotation_class in data_dict['annotation_class'][:21]]))

data: pd.DataFrame = pd.DataFrame(data_dict)

figure: plotly.graph_objects.Figure = plotly.express.bar(
    data_frame=data,
    x='text',
    y='amount',
    color='annotation_class',
    labels={
        "text": "texts",
        "amount": "annotation amount (in %)",
        "annotation_class": "annotation classes"
    },
    title=f'CS1 annotation amounts inside the initial chapters with {token_selection[1] - token_selection[0]} tokens <br><sub>(in percentage of the selected token amount of the respective chapter)</sub>',
    color_discrete_map={
        'Place-Container': '#B6D3FF',
        'Place-Container-MC': '#CCDEFF',
        'Place-Object': '#D4EAFF',
        'Place-Object-MC': '#E6F2FF',
        'Place-Abstract': '#89A8F6',
        'Place-Abstract-MC': '#98C3FA',
        'Place-ALT': '#90A6C7',
        'Movement-Subject': '#FF6D6D',
        'Movement-Object': '#F60D00',
        'Movement-Sound': '#FF4949',
        'Movement-Light': '#CA0B0B',
        'Movement-Smell': '#B60000',
        'Movement-ALT': '#960000',
        'Direction': '#92FFBD',
        'Direction-ALT': '#75CC96',
        'Positioning': '#DB8300',
        'Positioning-ALT': '#B56A01',
        'Dimensioning-Distance': '#8AB6AD',
        'Dimensioning-Size': '#7CD3C0',
        'Dimensioning-Amount': '#7EF5D9',
        'Dimensioning-ALT': '#60847B'
    },
    width=1400,
    height=800
)

figure.update_layout(font={'size': 18})

if get_bar_charts_with_annotation_amounts_of_all_chapters__show_html:
    figure.show()

if get_bar_charts_with_annotation_amounts_of_all_chapters__save_to_html:
    if not (os.path.isdir('perform_analysis_output')):
        os.makedirs('perform_analysis_output')
    figure.write_html(os.path.join('perform_analysis_output', f'cs1_annotation_amounts__{token_selection[1] - token_selection[0]}_tokens.html'))


#### get first character event overview

In [7]:
# install and import Pillow
%pip install Pillow==11.2.1
from PIL import Image, ImageDraw, ImageFont

Note: you may need to restart the kernel to use updated packages.


In [8]:
# load first character events data
first_character_events_data: pd.DataFrame = pd.read_csv(filepath_or_buffer=os.path.join('novel_beginning_analysis', 'categorization.tsv'), sep='\t')
first_character_events_data.head(8)

,text_id,grammatical_person,narrator,discourse,story,first_character_event_sentence,chapters_total_sentences,first_character_event_token,chapters_total_token,first_character_event_description,general_entrance_description
0,DEU19_030,3rd,heterodiegetic,high expositionality,in medias res,37.0,315,955.0,7179,professor looks at a manuscript,The opening is a spatial description of a nigh...
1,DEU19_001,3rd,heterodiegetic,high expositionality,in medias res,13.0,206,430.0,5491,a man steers a horse-drawn carriage,The chapter opens with a spatial description o...
2,DEU20_002,1st,homodiegetic,medium expositionality,in medias res,1.0,86,10.0,2689,arrival in Bonn,The chapter begins immediately with the arriva...
3,DEU20_021,3rd,heterodiegetic,medium expositionality,in medias res,1.0,43,13.0,744,Mr. B. buys a car,The chapter immediatly starts with the act of ...
4,SPA19_001,3rd,heterodiegetic,high expositionality,in medias res,1.0,54,18.0,1842,travel of three knights,The plot starts immediately by three knights c...
5,SPA19_008,3rd,heterodiegetic,medium expositionality,in medias res,12.0,158,401.0,4300,inhabitant of the tower walks around inside,The novel starts with spatial descriptions of ...
6,LAT19_004,3rd,heterodiegetic,medium expositionality,in medias res,1.0,37,17.0,1210,travel of indigenous man and women through val...,"In the year 1656, two travellers - an Andalusi..."
7,LAT19_041,3rd,heterodiegetic,high expositionality,ab ovo,NaN,27,NaN,1074,no character-related event in the first chapter,"The story is set in 1814, in the midst of the ..."


In [33]:
# colors
black = (0, 0, 0)
white = (255, 255, 255)
blueish = (52, 101, 164)
grey = (120, 120, 120)

# create first character event overview image
image = Image.new(mode='RGB', size=(1100, 1000), color=white)
draw = ImageDraw.Draw(im=image)

# border
draw.rectangle(xy=(25, 25, 1075, 975), fill=None, outline=black)

# text names
font_text_name = ImageFont.truetype(os.path.join('assets', 'fonts', 'Aspekta-400.ttf'), 22)
for index, row in first_character_events_data.iterrows():
    draw.text(xy=(50, 150 + (index * 95)), text=row['text_id'], fill=black, font=font_text_name)

# horizontal blue lines
for index, row in first_character_events_data.iterrows():
    draw.line(xy=(190, 160 + (index * 95), 860, 160 + (index * 95)), fill=blueish, width=2)

# arrow heads
for index, row in first_character_events_data.iterrows():
    draw.polygon(xy=[(860, 150 + (index * 95)), (860, 170 + (index * 95)), (880, 160 + (index * 95))], fill=blueish)

# vertical line next to text names
draw.line(xy=(190, 130, 190, 860), fill=black, width=2)

# token amount text
for index, row in first_character_events_data.iterrows():
    font_token_amount = ImageFont.truetype(os.path.join('assets', 'fonts', 'Aspekta-400.ttf'), 18)
    draw.text(xy=(930, 150 + (index * 95)), text=f'{row["chapters_total_token"]} Token', fill=grey, font=font_token_amount)

# vertical dotted line for middle of chapter
for z in range(90, 900, 10):
    draw.line(xy=(800, z, 800, z + 5), fill=black, width=2)

# rectangle sign middle of chapter
draw.rectangle(xy=(705, 40, 895, 90), fill=None, outline=black, width=2)
font_sign_middle_of_chapter = ImageFont.truetype(os.path.join('assets', 'fonts', 'Aspekta-400.ttf'), 18)
draw.text(xy=(725, 53), text='middle of chapter 1', fill=black, font=font_sign_middle_of_chapter)

# rectangle sign 1st character event example
draw.rectangle(xy=(405, 40, 595, 90), fill=None, outline=grey, width=2)
font_sign_1st_character_event_example = ImageFont.truetype(os.path.join('assets', 'fonts', 'Aspekta-400.ttf'), 18)
draw.text(xy=(425, 53), text='1st character event', fill=grey, font=font_sign_1st_character_event_example)
draw.line(xy=(393, 40, 393, 90), fill=black, width=3)

# draw 1st character event positions
for index, row in first_character_events_data.iterrows():
    # 800 - 190 = 610 (1st half of chapter length)
    # 1220 (complete chapter length)
    if math.isnan(row['first_character_event_token']):
        font_no_character_event_in_1st_chapter = ImageFont.truetype(os.path.join('assets', 'fonts', 'Aspekta-400.ttf'), 18)
        draw.text(xy=(325, 140 + (index * 95) + 30), text='no character event inside the 1st chapter', fill=grey, font=font_no_character_event_in_1st_chapter)
        continue
    x_pos = ((row['first_character_event_token'] / row['chapters_total_token']) * 1220) + 190
    draw.line(xy=(x_pos, 140 + (index * 95), x_pos, 180 + (index * 95)), fill=black, width=3)
    draw.rectangle(xy=(x_pos + 12, 140 + (index * 95), x_pos + 80, 180 + (index * 95)), fill=white, outline=grey, width=2)
    font_1st_chapter_event_token = ImageFont.truetype(os.path.join('assets', 'fonts', 'Aspekta-400.ttf'), 18)
    offset = 30 if int(row['first_character_event_token']) > 99 else 36
    draw.text(xy=(x_pos + offset, 148 + (index * 95)), text=str(int(row['first_character_event_token'])), fill=grey, font=font_1st_chapter_event_token)

# show and export
if get_first_character_event_overview__show_temporary_image:
    image.show()

if get_first_character_event_overview__save_to_png:
    if not (os.path.isdir('perform_analysis_output')):
        os.makedirs('perform_analysis_output')
    image.save(os.path.join('perform_analysis_output', 'first_character_event_overview.png'))


#### get first character event cs1 relations